<a href="https://colab.research.google.com/github/erokemwa/Data-Science/blob/main/Price_Monitoring_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install MetaTrader5

Imports the required libraries and sets up the constants. This should be run first to ensure all necessary modules are imported.
---



In [ ]:
```python
import MetaTrader5 as mt5
import time
import logging

# Constants
SYMBOL = "EURUSD"
PRICE_DELTA = 0.00005
CHECK_INTERVAL = 5  # seconds

Configures logging which will be used for outputting information, warnings, or errors throughout the script.

In [ ]:
```python
# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

Defines helper functions for initializing MetaTrader5, fetching price information, and monitoring the price. Each function is encapsulated for clarity and reusability.

In [ ]:
```python
def initialize_mt5():
    """Initialize MetaTrader5 and select the trading symbol."""
    if not mt5.initialize():
        logging.error("mt5.initialize() failed")
        return False
    if not mt5.symbol_select(SYMBOL, True):
        logging.error(f"Failed to select the symbol: {SYMBOL}")
        mt5.shutdown()
        return False
    return True

Function to get price information

In [ ]:
```python
def get_price_info():
    """Fetch the price information for the symbol."""
    price_info = mt5.symbol_info_tick(SYMBOL)
    if price_info is None:
        logging.error(f"Failed to get price info for {SYMBOL}")
        return None
    return price_info

Function to monitor price

In [ ]:
```python
def monitor_price(starting_price, price_level_high, price_level_low):
    """Monitor the price and check if it goes beyond set thresholds."""
    old_price = starting_price
    while True:
        price_info = get_price_info()
        if price_info is None:
            continue

        current_price = price_info.bid
        if current_price != old_price:
            logging.info(f"Current price of {SYMBOL} is {current_price}")
            old_price = current_price

        if current_price > price_level_high:
            logging.info(f"Current price of {SYMBOL} has reached the upper price limit of {price_level_high}")
            return True, current_price
        elif current_price < price_level_low:
            logging.info(f"Current price of {SYMBOL} has reached the lower price limit of {price_level_low}")
            return False, current_price

        time.sleep(CHECK_INTERVAL)

Contains the main logic of the script where we initialize, fetch initial price info, set price thresholds, and monitor the price.

In [ ]:
```python
def main():
    """Main function to run the trading bot."""
    if not initialize_mt5():
        return

    price_info = get_price_info()
    if price_info is None:
        mt5.shutdown()
        return

    starting_price = price_info.bid
    price_level_high = starting_price + PRICE_DELTA
    price_level_low = starting_price - PRICE_DELTA

    logging.info(f"The starting price is {starting_price}")
    logging.info(f"The high price is {price_level_high}")
    logging.info(f"The low price is {price_level_low}")

    # Monitor price and get result
    is_upper_limit, final_price = monitor_price(starting_price, price_level_high, price_level_low)

    # Here you can add your trading logic based on the result
    if is_upper_limit:
        logging.info(f"Price went above the upper limit. Final price: {final_price}")
    else:
        logging.info(f"Price went below the lower limit. Final price: {final_price}")

    # Shutdown MetaTrader5
    mt5.shutdown()

The entry point of the script when run as a standalone program. In a Jupyter Notebook, you would typically run this cell after defining all functions to execute the main logic.

In [ ]:
```python
if __name__ == "__main__":
    main()